In [1]:
import pandas as pd

file_path = 'D:/Data STATBIS ITS/TUGAS AKHIR/April-July2021_Sunda.csv'

# Membaca file CSV ke dalam DataFrame
data_frame = pd.read_csv(file_path)

# Menghitung jumlah baris data
jumlah_baris = data_frame.shape[0]

# Menampilkan jumlah baris
print("Jumlah baris data:", jumlah_baris)

Jumlah baris data: 7768690


In [2]:
jumlah_kapal = data_frame['msg_mmsi'].nunique()
print("Jumlah kapal:", jumlah_kapal)

Jumlah kapal: 3013


In [2]:
print("Daftar kolom:")
print(data_frame.columns)

Daftar kolom:
Index(['msg_mmsi', 'msg_kind', 'msg_repeat', 'data_lat', 'data_lon',
       'data_nav', 'data_rot', 'data_sog', 'data_cog', 'data_head', 'data_utc',
       'dt_last', 'ship_name', 'ship_csign', 'shipkind_id', 'shipflag_id',
       'ship_gt', 'ship_dwt', 'ship_lbp', 'ship_loa', 'ship_draft',
       'ship_depth', 'ship_beam', 'ship_build_int', 'ship_build_str',
       'ship_class', 'shipkind_name', 'flag_name\', 'dt_date time1',
       'dt_date time2'],
      dtype='object')


In [3]:
# Konversi kolom dt_date.time1 menjadi tipe datetime
data_frame['dt_date.time1'] = pd.to_datetime(data_frame['dt_date time1'])

In [4]:
# Langkah 1: Filter berdasarkan tanggal
start_date = pd.to_datetime('2021-05-01')
end_date = pd.to_datetime('2021-05-30')
filtered_data1 = data_frame[(data_frame['dt_date.time1'] >= start_date) & (data_frame['dt_date.time1'] <= end_date)]

# Menampilkan summary data dan jumlah data
print("Summary data setelah langkah 1:")
print(filtered_data1.describe())
print("Jumlah data setelah langkah 1:", filtered_data1.shape[0])

jumlah_kapal1 = filtered_data1['msg_mmsi'].nunique()
print("Jumlah kapal:", jumlah_kapal1)

Summary data setelah langkah 1:
           msg_mmsi      msg_kind  msg_repeat      data_lat      data_lon  \
count  3.216862e+06  3.216862e+06   3216862.0  3.216862e+06  3.216862e+06   
mean   5.148521e+08  2.010783e+00         0.0 -5.901589e+00  1.060392e+02   
std    5.521730e+07  3.694807e+00         0.0  1.224679e-01  2.197182e-01   
min    1.000000e+08  1.000000e+00         0.0 -7.053450e+00  1.046311e+02   
25%    5.250054e+08  1.000000e+00         0.0 -5.957500e+00  1.059682e+02   
50%    5.250171e+08  1.000000e+00         0.0 -5.934120e+00  1.059874e+02   
75%    5.251006e+08  1.000000e+00         0.0 -5.877890e+00  1.061081e+02   
max    8.686149e+08  1.900000e+01         0.0 -5.087170e+00  1.074605e+02   

           data_nav      data_rot      data_sog      data_cog     data_head  \
count  3.216862e+06  3.216862e+06  3.216862e+06  3.216862e+06  3.216862e+06   
mean   2.351376e+00 -7.440752e+01  3.666923e+00  1.773420e+02  3.678363e+02   
std    4.546512e+00  6.605966e+01  6.

In [5]:
# Langkah 2: Hapus pesan AIS dengan kecepatan tidak realistis (SOG ≥ 30 knots)
# Mengurutkan data berdasarkan MMSI dan waktu
filtered_data1 = filtered_data1.sort_values(by=['msg_mmsi', 'dt_date.time1'])

filtered_data2 = filtered_data1[filtered_data1['data_sog'] < 30]

filtered_data3 = filtered_data2[filtered_data2['data_nav'] != '0']

# Menampilkan summary data dan jumlah data
print("Summary data setelah langkah 3:")
print(filtered_data3.describe())
print("Jumlah data setelah langkah 3:", filtered_data3.shape[0])

jumlah_kapal3 = filtered_data3['msg_mmsi'].nunique()
print("Jumlah kapal:", jumlah_kapal3)

Summary data setelah langkah 3:
           msg_mmsi      msg_kind  msg_repeat      data_lat      data_lon  \
count  3.212055e+06  3.212055e+06   3212055.0  3.212055e+06  3.212055e+06   
mean   5.148224e+08  2.011761e+00         0.0 -5.902247e+00  1.060390e+02   
std    5.524108e+07  3.696904e+00         0.0  1.211018e-01  2.196997e-01   
min    1.000000e+08  1.000000e+00         0.0 -7.053450e+00  1.046311e+02   
25%    5.250054e+08  1.000000e+00         0.0 -5.957530e+00  1.059681e+02   
50%    5.250171e+08  1.000000e+00         0.0 -5.934260e+00  1.059873e+02   
75%    5.251006e+08  1.000000e+00         0.0 -5.878030e+00  1.061078e+02   
max    8.686149e+08  1.900000e+01         0.0 -5.087170e+00  1.074605e+02   

           data_nav      data_rot      data_sog      data_cog     data_head  \
count  3.212055e+06  3.212055e+06  3.212055e+06  3.212055e+06  3.212055e+06   
mean   2.353607e+00 -7.451501e+01  3.520818e+00  1.770727e+02  3.681378e+02   
std    4.549450e+00  6.604560e+01  4.

In [6]:
import pandas as pd
import numpy as np

# Menghitung selisih waktu antara dua pesan AIS berturut-turut
filtered_data3['time_diff'] = filtered_data3.groupby('msg_mmsi')['dt_date.time1'].diff()

# Menentukan selisih waktu maksimum
max_time_diff = pd.Timedelta(hours=2)

# Membuat kolom baru untuk menandai perjalanan yang berkesinambungan
filtered_data4 = filtered_data3.copy()
filtered_data4['continuous_trip'] = np.where(filtered_data4['time_diff'] <= max_time_diff, 0, 1)

# Mengatur nilai continuous_trip menjadi 0 untuk data pertama tiap kapal
first_index = filtered_data4.groupby('msg_mmsi').head(1).index
filtered_data4.loc[first_index, 'continuous_trip'] = 0

# Menampilkan summary data dan jumlah data
print(filtered_data4)
print("Jumlah data setelah langkah 4:", filtered_data4.shape[0])

          msg_mmsi  msg_kind  msg_repeat  data_lat   data_lon  data_nav  \
895059   100000000         1           0  -6.09692  106.88676         0   
898458   100000000         1           0  -6.08923  106.88496         0   
912263   100000000         1           0  -6.07502  106.88165         0   
917635   100000000         1           0  -6.10105  106.88769         0   
2034281  100000000         3           0  -5.95531  105.98379        15   
...            ...       ...         ...       ...        ...       ...   
3411694  810016036        18           0  -5.97872  106.11848         0   
3412086  810016036        18           0  -5.98180  106.11608         0   
3412624  810016036        18           0  -5.97851  106.11576         0   
3413271  810016036        18           0  -5.96020  106.11467         0   
2497260  868614888         1           0  -5.81522  106.10093         0   

         data_rot  data_sog  data_cog  data_head  ...  ship_build_int  \
895059     -128.0       0.

In [7]:
# Menghitung jumlah data dengan kategori 0 dan 1 pada kolom 'continuous_trip'
count_0 = filtered_data4[filtered_data4['continuous_trip'] == 0].shape[0]
count_1 = filtered_data4[filtered_data4['continuous_trip'] == 1].shape[0]

print("Jumlah data dengan kategori 0:", count_0)
print("Jumlah data dengan kategori 1:", count_1)

Jumlah data dengan kategori 0: 3204959
Jumlah data dengan kategori 1: 7096


In [8]:
# Langkah 5: Hapus kapal dengan kurang dari 20 baris data AIS
min_data_count = 30

# Menghitung jumlah baris data AIS per kapal
ais_counts = filtered_data4['msg_mmsi'].value_counts()

# Membuat daftar MMSI kapal yang memiliki setidaknya 20 baris data AIS
valid_mmsi_list = ais_counts[ais_counts >= min_data_count].index.tolist()

# Menggunakan daftar MMSI valid untuk memfilter data
filtered_data5 = filtered_data4[filtered_data4['msg_mmsi'].isin(valid_mmsi_list)]

# Menampilkan summary data dan jumlah data
print("Summary data setelah langkah 5:")
print(filtered_data5.describe())
print("Jumlah data setelah langkah 5:", filtered_data5.shape[0])

jumlah_kapal5 = filtered_data5['msg_mmsi'].nunique()
print("Jumlah kapal:", jumlah_kapal5)

Summary data setelah langkah 5:
           msg_mmsi      msg_kind  msg_repeat      data_lat      data_lon  \
count  3.209578e+06  3.209578e+06   3209578.0  3.209578e+06  3.209578e+06   
mean   5.148195e+08  2.006721e+00         0.0 -5.902309e+00  1.060386e+02   
std    5.523480e+07  3.686968e+00         0.0  1.209527e-01  2.190257e-01   
min    1.234568e+08  1.000000e+00         0.0 -7.053450e+00  1.046311e+02   
25%    5.250054e+08  1.000000e+00         0.0 -5.957520e+00  1.059681e+02   
50%    5.250171e+08  1.000000e+00         0.0 -5.934280e+00  1.059873e+02   
75%    5.251006e+08  1.000000e+00         0.0 -5.878070e+00  1.061078e+02   
max    8.100160e+08  1.900000e+01         0.0 -5.087170e+00  1.074605e+02   

           data_nav      data_rot      data_sog      data_cog     data_head  \
count  3.209578e+06  3.209578e+06  3.209578e+06  3.209578e+06  3.209578e+06   
mean   2.354232e+00 -7.454517e+01  3.519801e+00  1.770780e+02  3.681310e+02   
std    4.550327e+00  6.604206e+01  4.

In [9]:
min_trip_duration = pd.Timedelta(hours=4)
filtered_data6 = filtered_data5.groupby('msg_mmsi').filter(lambda x: x['time_diff'].sum() >= min_trip_duration)

# Menampilkan summary data dan jumlah data
print("Summary data setelah langkah 6:")
print(filtered_data6.describe())
print("Jumlah data setelah langkah 6:", filtered_data6.shape[0])
jumlah_kapal6 = filtered_data6['msg_mmsi'].nunique()
print("Jumlah kapal:", jumlah_kapal6)

Summary data setelah langkah 6:
           msg_mmsi      msg_kind  msg_repeat      data_lat      data_lon  \
count  3.198727e+06  3.198727e+06   3198727.0  3.198727e+06  3.198727e+06   
mean   5.148560e+08  2.001954e+00         0.0 -5.902520e+00  1.060386e+02   
std    5.506255e+07  3.676436e+00         0.0  1.204483e-01  2.179279e-01   
min    1.234568e+08  1.000000e+00         0.0 -7.053450e+00  1.046311e+02   
25%    5.250054e+08  1.000000e+00         0.0 -5.957510e+00  1.059683e+02   
50%    5.250171e+08  1.000000e+00         0.0 -5.934440e+00  1.059873e+02   
75%    5.251006e+08  1.000000e+00         0.0 -5.878320e+00  1.061078e+02   
max    8.100160e+08  1.900000e+01         0.0 -5.087170e+00  1.074605e+02   

           data_nav      data_rot      data_sog      data_cog     data_head  \
count  3.198727e+06  3.198727e+06  3.198727e+06  3.198727e+06  3.198727e+06   
mean   2.360276e+00 -7.473334e+01  3.491650e+00  1.771130e+02  3.685284e+02   
std    4.554958e+00  6.601938e+01  4.

In [10]:
# Langkah 7: Menghitung kecepatan empiris dan hapus pesan abnormal dengan kecepatan empiris di atas 40 knots
import numpy as np

# Mengurutkan data berdasarkan MMSI dan waktu
filtered_data7= filtered_data6.sort_values(by=['msg_mmsi', 'dt_date.time1'])

# Menghitung jarak antara dua titik koordinat menggunakan haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius bumi dalam kilometer
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Menghitung jarak antara dua pesan AIS
filtered_data7['distance'] = haversine(filtered_data7['data_lat'].shift(), filtered_data7['data_lon'].shift(), filtered_data7['data_lat'], filtered_data7['data_lon'])

# Menghapus data dengan selisih waktu NaN
filtered_data7 = filtered_data7.dropna(subset=['time_diff'])

# Mengonversi timedelta menjadi jumlah detik (integer)
filtered_data7['time_diff_seconds'] = filtered_data7['time_diff'].dt.total_seconds()

# Menghitung kecepatan empiris (dalam kilometer per jam)
filtered_data7['empirical_speed'] = filtered_data7['distance'] / ((filtered_data7['time_diff_seconds'] / 3600))

# Menghapus pesan dengan kecepatan empiris di atas 40 knots (74.08 km/jam)
filtered_data8 = filtered_data7[filtered_data7['empirical_speed'] < 40*1.852]

# Menampilkan summary data dan jumlah data
print("Jumlah data setelah filtering:", filtered_data8.shape[0])
jumlah_kapal8 = filtered_data8['msg_mmsi'].nunique()
print("Jumlah kapal:", jumlah_kapal8)

Jumlah data setelah filtering: 1392552
Jumlah kapal: 1403


In [11]:
# Menghitung jumlah data masing-masing kapal sebelum preprocessing
jumlah_data_sebelum = filtered_data7['msg_mmsi'].value_counts().reset_index()
jumlah_data_sebelum.columns = ['msg_mmsi', 'jumlah_data_sebelum']

# Menghitung jumlah data masing-masing kapal setelah preprocessing
jumlah_data_sesudah = filtered_data8['msg_mmsi'].value_counts().reset_index()
jumlah_data_sesudah.columns = ['msg_mmsi', 'jumlah_data_sesudah']

# Menggabungkan data jumlah data sebelum dan sesudah preprocessing
data_perubahan = jumlah_data_sebelum.merge(jumlah_data_sesudah, on='msg_mmsi', how='outer')

# Menampilkan data perubahan
print(data_perubahan)

       msg_mmsi  jumlah_data_sebelum  jumlah_data_sesudah
0     525005357                77532                17709
1     525019545                72029                19084
2     525000033                65473                16399
3     525017090                56091                17655
4     525007232                49812                17621
...         ...                  ...                  ...
1398  525016441                   31                   31
1399  477669400                   31                   22
1400  525015534                   30                   23
1401  525400142                   29                   27
1402  525020293                   29                   27

[1403 rows x 3 columns]


In [12]:
import pandas as pd

# Mengurutkan data berdasarkan MMSI dan waktu
filtered_data8 = filtered_data8.sort_values(by=['msg_mmsi', 'dt_date.time1'])

# Menambahkan kolom baru 'previous_time' yang berisi waktu sebelumnya untuk setiap kapal
filtered_data8['previous_time'] = filtered_data8.groupby('msg_mmsi')['dt_date.time1'].shift(1)

# Menghitung selisih waktu antara waktu saat ini dan waktu sebelumnya (dalam detik)
filtered_data8['time_diff'] = (filtered_data8['dt_date.time1'] - filtered_data8['previous_time']).dt.total_seconds()

# Mengidentifikasi data yang berada dalam interval waktu yang sama (dalam hal ini, 1 menit)
filtered_data8['is_duplicate_within_1_minute'] = (filtered_data8['time_diff'] <= 60) & (filtered_data8['time_diff'] >= 0)

# Memfilter data untuk hanya menyertakan data yang bukan duplikat dalam 1 menit
filtered_data9 = filtered_data8[~filtered_data8['is_duplicate_within_1_minute']]

# Menghapus kolom-kolom tambahan yang tidak diperlukan lagi
filtered_data9 = filtered_data9.drop(columns=['previous_time', 'time_diff', 'is_duplicate_within_1_minute'])

# Menampilkan summary data dan jumlah data
print("Jumlah data setelah filtering:", filtered_data9.shape[0])
jumlah_kapal9 = filtered_data9['msg_mmsi'].nunique()
print("Jumlah kapal:", jumlah_kapal9)


Jumlah data setelah filtering: 568670
Jumlah kapal: 1403


In [14]:
import pandas as pd

# Simpan filtered_data7 ke dalam file CSV
filtered_data9.to_csv('preprocessed_568670.csv', index=False)

In [15]:
# Simpan filtered_data ke dalam file CSV
filtered_data8.to_csv('preprocessed_1392552.csv', index=False)

In [13]:
# Menghitung jumlah data masing-masing kapal sebelum preprocessing
jumlah_data_sebelum = filtered_data8['msg_mmsi'].value_counts().reset_index()
jumlah_data_sebelum.columns = ['msg_mmsi', 'jumlah_data_sebelum']

# Menghitung jumlah data masing-masing kapal setelah preprocessing
jumlah_data_sesudah = filtered_data9['msg_mmsi'].value_counts().reset_index()
jumlah_data_sesudah.columns = ['msg_mmsi', 'jumlah_data_sesudah']

# Menggabungkan data jumlah data sebelum dan sesudah preprocessing
data_perubahan = jumlah_data_sebelum.merge(jumlah_data_sesudah, on='msg_mmsi', how='outer')

# Menampilkan data perubahan
print(data_perubahan)

       msg_mmsi  jumlah_data_sebelum  jumlah_data_sesudah
0     525019545                19084                 2683
1     525005357                17709                 2949
2     525017090                17655                 3023
3     525007232                17621                 3385
4     525019409                16727                 3175
...         ...                  ...                  ...
1398  525006213                   23                   15
1399  477669400                   22                   19
1400  525010156                   20                   10
1401  564911000                   20                   14
1402  351621000                   18                   15

[1403 rows x 3 columns]
